<code>pip install retina-face</code>



In [39]:
import cv2
import os
import numpy as np
from retinaface import RetinaFace

In [40]:
def emotionImage(emotion):
	# Emojis
	if emotion == 'Felicidad': image = cv2.imread('./Emojis/Felicidad.png')
	if emotion == 'Enojo': image = cv2.imread('./Emojis/Enojado.png')
	if emotion == 'Sorpresa': image = cv2.imread('./Emojis/Sorpresa.png')
	if emotion == 'Tristeza': image = cv2.imread('./Emojis/Tristeza.png')
	if emotion == 'Neutro': image = cv2.imread('./Emojis/Neutro.png')
	return image

De más rápido a más lento:
 * LBPH (Reconoce bastante bien)
 * FisherFaces ()

In [41]:
emotion_recognizer = cv2.face.LBPHFaceRecognizer_create()
emotion_recognizer.read('./Modelos/modeloLBPH.xml')

In [42]:
dataPath = './fotos_entrenamiento/' #Cambia a la ruta donde hayas almacenado Data
imagePaths = os.listdir(dataPath)
print('imagePaths=',imagePaths)

imagePaths= ['Enojo', 'Felicidad', 'Neutro', 'Sorpresa', 'Tristeza']


In [47]:
cap = cv2.VideoCapture(0)

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

while True:

	ret,frame = cap.read()
	if ret == False: break
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	auxFrame = gray.copy()
 
	faces = faceClassif.detectMultiScale(gray,
									scaleFactor=1.1,
									minNeighbors=10,
									minSize=(30, 30),
								)

	for (x,y,w,h) in faces:
		rostro = auxFrame[y:y+h,x:x+w]
		rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
		result = emotion_recognizer.predict(rostro)

		cv2.putText(frame,'{}'.format(result),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
  
		if result[1] < 60:
			print(result)
			if(result[0] is 0): color = (50, 50, 255)
			if(result[0] is 1): color = (50, 255, 100)
			if(result[0] is 2): color = (255, 255, 255)
			if(result[0] is 3): color = (255, 255, 75)
			if(result[0] is 4): color = (255, 50, 75)
			cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(color),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),color,2)
			#nFrame = cv2.hconcat([frame,image])
		else:
			cv2.putText(frame,'No identificado',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
			#nFrame = cv2.hconcat([frame,np.zeros((480,300,3),dtype=np.uint8)])

	cv2.imshow('Vigila tus sentimientos',frame)
	k = cv2.waitKey(1)
	if k == 27:
		break

cap.release()
cv2.destroyAllWindows()

(1, 41.80715901755414)
(4, 42.268902420110244)
(2, 39.49053452654558)
(2, 37.34075343020296)
(2, 37.78591047594454)
(2, 38.290198779356665)
(2, 37.15583019019811)
(2, 36.60944960604886)
(2, 37.20615219443976)
(2, 36.5864020541693)
(2, 36.98239958666728)
(2, 37.41954740485319)
(2, 36.398993214404456)
(2, 37.204929721595214)
(1, 40.11746136554229)
(1, 41.42557489668726)
(1, 41.994958820964)
(1, 39.709641757026645)
(1, 41.55649532314299)
(1, 41.17269096023716)
(1, 43.951623613611055)
(1, 38.628521900348105)
(1, 38.75192205167385)
(1, 39.236786826808135)
(1, 38.75293453121802)
(2, 37.445838304415474)
(2, 40.55992809743113)
(4, 35.45185119770365)
(4, 37.50278592000299)
(4, 40.88968455096689)
(4, 37.50994129072563)
(4, 36.04667549460033)
(4, 36.482320641391915)
(4, 35.963741449621644)
(4, 37.36669695273455)
(4, 35.65504106717731)
(4, 36.49783074481376)
(4, 37.87698868568768)
(4, 36.739149240936264)
(4, 37.44141199076545)
(0, 37.002043086028806)
(2, 37.37965921036172)
(0, 37.749061510518)
(0,